In [3]:
from pywinauto import application
import time
import os

# 증권사 API 자동 실행
os.system('taskkill /IM coStarter* /F /T')
os.system('taskkill /IM CpStart* /F /T')
os.system('wmic process where "name like \'%coStarter%\'" call terminate')
os.system('wmic process where "name like \'%CpStart%\'" call terminate')
time.sleep(5)        

app = application.Application()
app.start('C:\CREON\STARTER\coStarter.exe /prj:cp /id:{id} /pwd:{pwd} /pwdcert:{pwd} /autostart'.format(
id='sheee7', pwd='sh5167!!'))
time.sleep(60)

In [4]:
import win32com.client

# 서버 접속 확인
objCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
bConnect = objCpCybos.IsConnect

if (bConnect == 1): #접속 - 1, 비접속 - 0
    print("서버가 정상적으로 접속되었습니다.")
else:
    print("서버가 정상적으로 연결되지 않았습니다. ")
    exit()

서버가 정상적으로 접속되었습니다.


In [1]:
import pandas as pd
#testdf = pd.read_excel('21.01.12_stock_data.xlsx')
#testdf = pd.read_excel('21.01.29_stock_data.xlsx')
testdf = pd.read_excel('21.02.05_stock_data.xlsx')

In [2]:
"""
# Colab에서 사용하기 위함
from google.colab import drive
drive.mount('/content/drive')

testdf = pd.read_excel('/content/drive/MyDrive/Programming/StockProject/21.02.05_stock_data.xlsx')
"""

"\n# Colab에서 사용하기 위함\nfrom google.colab import drive\ndrive.mount('/content/drive')\n\ntestdf = pd.read_excel('/content/drive/MyDrive/Programming/StockProject/21.02.05_stock_data.xlsx')\n"

In [3]:
"""
# Colbal에서 사용하기 위함
import pandas_datareader as pdr
from datetime import datetime

start = datetime(2015,1,1)
end = datetime(2021,2,10)

#(종목코드, 'yahoo')
df1 = pdr.DataReader('005930.KS', 'yahoo', start, end)
"""

"\n# Colbal에서 사용하기 위함\nimport pandas_datareader as pdr\nfrom datetime import datetime\n\nstart = datetime(2015,1,1)\nend = datetime(2021,2,10)\n\n#(종목코드, 'yahoo')\ndf1 = pdr.DataReader('005930.KS', 'yahoo', start, end)\n"

In [4]:
#import pandas as pd
import time
import win32com.client

# 종목명 -> 종목코드 변환
instCpStockCode = win32com.client.Dispatch("CpUtil.CpStockCode")
Stockname = input("종목명을 입력해주세요 : ")
per = int(input("적용할 PER을 입력해주세요 : "))
Stockcode = instCpStockCode.NameToCode(Stockname)
tablename = Stockname

now = time.strftime('%Y%m%d')

Stockpastday = '20150101'
Stockpresentday = now

# 오브젝트 가져오기
inStockMst = win32com.client.Dispatch("dscbo1.StockMst") #현재 미사용
inStockChart = win32com.client.Dispatch("CpSysDib.StockChart")

inStockChart.SetInputValue(0, Stockcode) # 종목 코드 - 삼성전자
inStockChart.SetInputValue(1, ord('1')) # 기간으로 확인
inStockChart.SetInputValue(2, Stockpresentday) # 최신일자
inStockChart.SetInputValue(3, Stockpastday) # 과거일자
inStockChart.SetInputValue(5, [0,2,3,4,5,8]) # 날짜,시가,고가,저가,종가,거래량
inStockChart.SetInputValue(6, ord('D')) # 차트 주가 - 일간 차트 요청
inStockChart.SetInputValue(9, ord('1')) # 수정주가 사용

inStockChart.BlockRequest()

len = inStockChart.GetHeaderValue(3)

data = []

for i in range(len):
    line = []  # 안쪽 리스트로 사용할 빈 리스트 생성
    date = inStockChart.GetDataValue(0, i)  # 날짜
    open = inStockChart.GetDataValue(1, i)  # 시가
    high = inStockChart.GetDataValue(2, i)  # 고가
    low = inStockChart.GetDataValue(3, i)  # 저가
    close = inStockChart.GetDataValue(4, i)  # 종가
    vol = inStockChart.GetDataValue(5, i)  # 거래량

    line.append(date)  # 리스트에 내용 추가
    line.append(open)
    line.append(high)
    line.append(low)
    line.append(close)
    line.append(vol)
    data.append(line)  # 전체 리스트에 안쪽 리스트를 추가

labels = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']

# 리스트 형태를 라벨이 있는 데이터프레임으로 변환하는 내용
df1 = pd.DataFrame.from_records(data, columns=labels)

종목명을 입력해주세요 : LG화학
적용할 PER을 입력해주세요 : 50


In [5]:
from datetime import datetime
# Date 일자를 문자열로 변환
df1['Date'] = df1['Date'].astype('str')
# datetime 를 이용해서 날짜로 변환
df1['Date'] = pd.to_datetime(df1['Date'])
#df1.info()

# 스크랩한 데이터에서 검색하려는 종목명과 일치하는 것을 찾아내기
sam = (testdf['종목명'] == Stockname)
testdf_sam = testdf[sam]
# 스크랩한 데이터에서 '2019/12'가 포함되는 것을 찾아내기
data2019 = (testdf['index'] == '2019/12')

#testdf_data2019 = testdf[data2019]

#data2021 = (testdf['발행주식수'] == '2021/12(E)')
#testdf_data2021 = testdf[data2021]

# 여러 조건에 부합되는 결과값 (& and // | or)
# '종목명'과 '2019/12'가 포함된 데이터만 찾기
testdf_total = testdf[sam & data2019]
#testdf_total

#testdf_sam
print(testdf_total)
# API를 통해서 최신 발행주식수를 가져오는 것이 'Best'이지만... 코드가 많이 추가되니 간단하게 넘어감..ㅋㅋ
# 스크랩한 데이터 중 발행주식수가 있으니 2019년의 발행주식수를 가져옴
stock_num = testdf_total['발행주식수']
stock_num = stock_num.values
print(stock_num)
print(testdf_sam['영업이익'])
# 나만의 목표가 계산식
testdf_sam['적정가격'] = round((testdf_sam['영업이익']*per *100000000 / (stock_num *1000) + (testdf_sam['EPS(원)']*per)) /2,-2)
print(testdf_sam['적정가격'])
#testdf_sam[['index', '적정가격']]

        index       매출액  영업이익  영업이익(발표기준)  당기순이익  지배주주순이익  비지배주주순이익    자산총계  \
7588  2019/12  286250.0  8956        8956   3761   3134.0     627.0  340244   

        부채총계    자본총계  ...   PBR  발행주식수  배당수익률   종목명  보험료수익  순이익률  순영업수익 이자수익  \
7588  166406  173838  ...  1.43  70592   0.63  LG화학    NaN   NaN    NaN  NaN   

     영업수익  level_0  
7588  NaN      NaN  

[1 rows x 33 columns]
[70592]
7584    18236
7585    19919
7586    29285
7587    22461
7588     8956
7589    23532
7590    36131
7591    40789
Name: 영업이익, dtype: int64
7584    1035900.0
7585    1138800.0
7586    1658500.0
7587    1265800.0
7588     417200.0
7589    1126200.0
7590    1973900.0
7591    2236200.0
Name: 적정가격, dtype: float64


C:\Users\azxsd\anaconda3\envs\venv_py37_32\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [6]:
#testdf_sam['index'] = testdf_sam['index'].astype('str')

# 적정가격을 차트상에 표시하기 위해 일자로 변경
# 단순작업으로 바꾸는 코드... 이기 때문에 2024, 2025가 되더라도 작동하도록 수정할 것
testdf_sam.loc[testdf_sam['index'] == '2020/12(P)','index'] ='2020/12'
testdf_sam.loc[testdf_sam['index'] == '2020/12(E)','index'] ='2020/12'
testdf_sam.loc[testdf_sam['index'] == '2021/12(E)','index'] ='2021/12'
testdf_sam.loc[testdf_sam['index'] == '2022/12(E)','index'] ='2022/12'
testdf_sam.loc[testdf_sam['index'] == '2023/12(E)','index'] ='2023/12'

# datetime을 이용해 년/월 형식으로 수정함 (2020/12 -> 2020-12-01)
testdf_sam['index'] = pd.to_datetime(testdf_sam['index'], format='%Y/%m')

from pandas.tseries.offsets import *

# 2019-12-01의 경우 일요일(주말)이라 차트상에 표시가 안되는 현상을 방지하고자, 일자를 바꿈
testdf_sam['index'] = testdf_sam['index'] + Week(weekday=4)

#testdf_sam[['index', '적정가격']]
#testdf_sam

C:\Users\azxsd\anaconda3\envs\venv_py37_32\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\azxsd\anaconda3\envs\venv_py37_32\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Users\azxsd\anaconda3\envs\venv_py37_32\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

In [7]:
import plotly.graph_objs as go
from datetime import datetime

# 현재주가 추출
nowprice = df1.loc[0, 'Close']
print(nowprice)
# 2021년 목표주가 추출(2022년 7)
tp21 = testdf_sam.iloc[6, 33]
tp22 = testdf_sam.iloc[7, 33]
# 2021년 영업이익 추출
op21 = testdf_sam.iloc[6, 2]
op22 = testdf_sam.iloc[7, 2]
# 목표주가 상승여력(퍼센트)
tpper21 = round(((tp21-nowprice)/nowprice*100),2)
tpper22 = round(((tp22-nowprice)/nowprice*100),2)

title = Stockname + ' 목표주가 / PER ' + str(per) +' / '+ str(nowprice) +'원('+ str(tp21) + '원 ' + str(round(tpper21,2)) + '%, '+str(op21)+'억)'
print(title)

fig = go.Figure(data=[go.Candlestick(x=df1['Date'],
                open=df1['Open'],
                high=df1['High'],
                low=df1['Low'],
                close=df1['Close'])])

fig.update_layout(
    margin=dict(l=20, r=20, t=60, b=20),
    title_text=title
)

fig.add_trace(go.Scatter(
    name="목표주가",
    mode="markers+lines", x=testdf_sam["index"], y=testdf_sam["적정가격"],
    xperiod="M1",
    xperiodalignment="middle"
))

"""
fig.add_hline(y=80000, line_dash="dot",
              annotation_text="8만 전자",
              annotation_position="bottom right",
              annotation_font_size=15)

fig.add_vrect(x0="2020-12-25", x1="2021-01-10", col=1,
              annotation_text="매수 기회", 
              annotation_position="top left",
              annotation_font_size=15, 
              annotation_font_color="red", 
              fillcolor="green", opacity=0.25, line_width=0)
"""

fig.update_xaxes(
    rangebreaks=[
        dict(bounds=["sat", "mon"]), #hide weekends
        dict(values=["2020-12-25", "2021-01-01"])  # hide Christmas and New Year's
    ],
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)

fig.show()


print('21년 목표주가 / PER %d / %d원(%f%%) %d억' 
%(per, tp21, tpper21, op21))
print('22년 목표주가 / PER %d / %d원(%f%%) %d억' 
%(per, tp22, tpper22, op22))

#print('%s 22년 목표주가 / PER %d / %d원(%f%%) %d억' %(Stockname, per, tp22, tpper22, op22))

831000
LG화학 목표주가 / PER 50 / 831000원(1973900.0원 137.53%, 36131억)


TypeError: 'NoneType' object is not callable